In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import heapq

In [14]:
#the following is loading testing dataset
filepath = '../../data/iptvdata/'
# filename = 'iptv_sample1000000.csv'
filename = 'iptv_sample123987.csv'
df = pd.read_csv(filepath+filename)
df = df.sort_values(by=['time'],ascending = (True))
df

,time,user,item
15360,2010-06-24 00:00:11,00111010000100010001011101010011,触电情缘(第15集).mp4
24887,2010-06-24 00:00:42,00111010000100010011001110101111,新三国(第29集).mp4
48925,2010-06-24 00:00:58,00111010000100010110110110100111,G4特工.mp4
15361,2010-06-24 00:01:01,00111010000100010001011101010011,触电情缘(第15集).mp4
54855,2010-06-24 00:01:16,00111010000100010111010010000110,算死草.mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [15]:
#dataset parameter:
dataset_begintime = datetime(2010,6,24)
ratio = 0.5
df['time'] = pd.to_datetime(df['time'])
splittime = dataset_begintime + (df['time'].max()- dataset_begintime)*ratio
df_test = df[df['time']>splittime]
df_test

,time,user,item
49590,2010-06-30 11:59:53,00111010000100010110110111101100,变形金刚(蓝光).mp4
12965,2010-06-30 12:00:14,00111010000100010001011101010010,鬼妈妈.mp4
49591,2010-06-30 12:00:23,00111010000100010110110111101100,变形金刚(蓝光).mp4
49592,2010-06-30 12:00:25,00111010000100010110110111101100,蓝色激情.mp4
2043,2010-06-30 12:00:41,00111010000100010000101011001011,那些迷人的往事(第28集).mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [16]:
df_test_earlymorning = df_test[df_test['time']>datetime(2010,7,5)]
df_test_earlymorning = df_test_earlymorning[df_test_earlymorning['time']<datetime(2010,7,5,7)]
def isseries(item):
    if len(item.split('(第')) >= 2 and len(item.split('(第')[1].split('集)')) >= 2 and item.split('(第')[1].split('集)')[0].isdigit():
        return True
    elif len(item)>3 and item[-1].isdigit() and not item[-4].isdigit() and item[-3]!='/' and item[-3]!=':' and item[-3:]!='mp4' and len(re.findall(r"\d+", item[-3:]))>0:
        return True
    else:
        return False

In [17]:
df_test_earlymorning

,time,user,item
26494,2010-07-05 00:01:05,00111010000100010011100101110101,神枪手(第09集).mp4
34790,2010-07-05 00:01:17,00111010000100010100101101010100,新三国(第65集).mp4
4140,2010-07-05 00:01:19,00111010000100010001001101001101,鼠胆龙威.mp4
34895,2010-07-05 00:01:51,00111010000100010100101101110001,致命摇篮.mp4
48158,2010-07-05 00:03:27,00111010000100010110110100101110,东方茱丽叶(第02集).mp4
...,...,...,...
3658,2010-07-05 06:39:03,00111010000100010001001011101101,亮剑(第28集).mp4
22405,2010-07-05 06:40:37,00111010000100010010110010010100,新三国(第53集).mp4
22406,2010-07-05 06:41:59,00111010000100010010110010010100,新三国(第53集).mp4
1262,2010-07-05 06:50:55,00111010000100010000100100111000,新三国(第15集).mp4


In [18]:
count_series = 0
total = 0
for index, row in df_test_earlymorning.iterrows():
    item = row[2]
    total += 1
    if isseries(item):
        count_series += 1
print(count_series/total)
    

0.5639810426540285


In [19]:
isseries(item)

True

In [20]:
df_test.dropna(inplace=True)
# np.nan in list(['item'])
df_test

C:\Users\lilee\anaconda3new\envs\temporal_3_6python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,time,user,item
49590,2010-06-30 11:59:53,00111010000100010110110111101100,变形金刚(蓝光).mp4
12965,2010-06-30 12:00:14,00111010000100010001011101010010,鬼妈妈.mp4
49591,2010-06-30 12:00:23,00111010000100010110110111101100,变形金刚(蓝光).mp4
49592,2010-06-30 12:00:25,00111010000100010110110111101100,蓝色激情.mp4
2043,2010-06-30 12:00:41,00111010000100010000101011001011,那些迷人的往事(第28集).mp4
...,...,...,...
4084,2010-07-06 23:58:23,00111010000100010001001100110100,虎胆龙威4.mp4
24028,2010-07-06 23:58:45,00111010000100010011001001111000,黑执事(第06集).mp4
34829,2010-07-06 23:58:50,00111010000100010100101101011100,全职杀手.mp4
38871,2010-07-06 23:59:25,00111010000100010101001101001000,包三姑外传(第01集).mp4


In [21]:
def get_min_lfu_score_from_cache(your_cache,lfu_score):
    kick_name = None
    min_score = 10000000
    for content_name in your_cache:
        if lfu_score[content_name] < min_score:
            min_score = lfu_score[content_name]
            kick_name = content_name    
    return kick_name    

In [22]:
def get_topk_lfu_content(topk_predictive_rank,LFU_score):
    return heapq.nlargest(topk_predictive_rank, LFU_score, key= lambda x:LFU_score[x])

In [23]:
# dicta = {'a':1,'b':9,'c':3}
# # type(dicta)
# heapq.nlargest(2, dicta, key= lambda x:dicta[x])

In [24]:
def caching_simulation_lfu(CACHE_SIZE = 500,N=1,LFU_win = 5000, downloadtime = 0.5, topk_predictive_rank = 1000, printoutfrequency = 5000,begintime = datetime(2010,7,5),stop_time = datetime(2010,7,7)):
#     ngram_dict = {}
#     ngram_dict_confidence = {}
    total_count = 0
    local_count = 0
    your_cache = set()
    hit = 0
    accum_hit = 0
    soft_hit = 0
    proactive_download = 0
    reactive_download_number =0
    hit_rate_list = []
    x_time_list = []
    req_rate_list = []
    utilization_list_reactive = []
    past_timestamps = {}
    
    
    LFU_score = {}
    LFU_queue = []
    
    
    topk_content_with_lfu_score = set()
    hit_in_near_future = 0
    hit_in_near_future_accum = 0
    hit_in_near_future_list = []
    
    
    reactive_latency = 0
    proactive_latency = 0
    calculatetime = begintime
    printouttime = begintime
    bandwidth_link = {'begin time':[],'end time':[],'item':[],'proactive_indicator':[]}
    print('cache size:',CACHE_SIZE, "LFU win", LFU_win,'downloadtime',downloadtime,'begintime',begintime,'stoptime',stop_time)
    for index, row in df_test.iterrows():
        user = row[1]
        time = row[0].to_pydatetime()
#         time = datetime(2010,6,24) + timedelta(seconds=time)
#         print(time)
    #         location = row[2]
        item = row[2]
#         if len(item.split('(第'))>=2:
#     #         print(item)
#             pass
#         else:
#             continue
        if time < begintime:
            continue
        #initial link empty time is the begin time
        if total_count == 0:
            link_empty_time = time

        total_count += 1
        local_count += 1
#-------------------------------------------------------------------------------------------------------------------
        #update link
        for i in range(len(bandwidth_link['item'])):
            if bandwidth_link['end time'][0] < time:
                req_time = bandwidth_link['begin time'].pop(0)
                downloaoded_time = bandwidth_link['end time'].pop(0)
                downloaded_item = bandwidth_link['item'].pop(0)
                proactive_indicator = bandwidth_link['proactive_indicator'].pop(0)

                if len(your_cache) < CACHE_SIZE:
                    #add item to cache
                    your_cache.add(downloaded_item)

                else:
                    #update cache

                    your_cache.add(downloaded_item)
                    kick_name = get_min_lfu_score_from_cache(your_cache,LFU_score)
                    your_cache.remove(kick_name)


                if proactive_indicator is False:
                    # for proactive latency, we calculate that in other part.
                    # get reactive latency
                    reactive_latency += (downloaoded_time - req_time).total_seconds()
                    # print((downloaoded_time - req_time).total_seconds())
                    if reactive_latency<0:
                        print('error')
#-------------------------------------------------------------------------------------------------------------------
        if item not in past_timestamps:
            past_timestamps[item] = [time]
        else:
            past_timestamps[item].append(time)

#-------------------------------------------------------------------------------------------------------------------
        if item in your_cache:
            hit += 1
            accum_hit += 1
            # deffect(time_window,time,user,name,scores)
        elif item in bandwidth_link['item']:
            soft_hit += 1
        else:
            # not hit and download
            bandwidth_link['begin time'].append(time)
            bandwidth_link['end time'].append(max(link_empty_time,time) + timedelta(seconds=downloadtime))
            bandwidth_link['item'].append(item)
            bandwidth_link['proactive_indicator'].append(False)
            reactive_download_number += 1
            link_empty_time = max(link_empty_time,time) + timedelta(seconds=downloadtime)

            
# ----------------------------------------------------------------------------------------------------------
# assign score
        if item not in LFU_score:
            LFU_score[item] = 1
        else:
            LFU_score[item] += 1
        LFU_queue.append(item)
        if len(LFU_queue)>LFU_win:
            lfupopitem = LFU_queue.pop(0)            
            LFU_score[lfupopitem] -= 1                                                                                                                                        


# ---------------------------------------------------------------------------------------count hit in near future       
        if item in topk_content_with_lfu_score:
            hit_in_near_future += 1
            hit_in_near_future_accum += 1


        if total_count % printoutfrequency == 0:
#         if time >= printouttime + printoutfrequency :
#             printouttime += printoutfrequency               
# -----------------------------------------------near future hit count            
            topk_content_with_lfu_score = get_topk_lfu_content(topk_predictive_rank,LFU_score)
            hit_in_near_future_list.append(hit_in_near_future/local_count)
            print('hit in near future',round(hit_in_near_future/local_count,4))
            hit_in_near_future = 0
        
# ------------------------------------------------------------        
        
            timedelta_frequency = (time - calculatetime).total_seconds()

#             print('delta',timedelta_frequency)
            reactive_number_in_bandwidth = 0
            proactive_number_in_bandwidth = 0
            for i in range(len(bandwidth_link['item'])):
                if bandwidth_link['proactive_indicator'][i]:
                    proactive_number_in_bandwidth += 1
                else:
                    reactive_number_in_bandwidth += 1

#             print('reactive donwload number',reactive_download_number)
#             print('proactive donwload number',proactive_download)
#                 print('reactive in link',reactive_number_in_bandwidth)
#                 print('roactive in link',proactive_number_in_bandwidth)
            reactive_utilization = round((reactive_download_number - reactive_number_in_bandwidth)*downloadtime / timedelta_frequency,4)
            utilization_list_reactive.append(reactive_utilization)
#             print(reactive_utilization)
            
            
            
            hit_rate_list.append(hit/local_count)
            x_time_list.append(str(time.day) +'_'+ str(time.hour))
            req_rate = round(local_count/(time - calculatetime).total_seconds()*60,2)
            calculatetime = time
            req_rate_list.append(req_rate)
            print(round(hit / local_count, 4),'proactive download',proactive_download,soft_hit,'buffer content',len(bandwidth_link['item']))
            
            hit = 0
            local_count = 0
            proactive_download = 0
            reactive_download_number =0
            soft_hit = 0
            print(time)
#             print(req_rate)
            

#         if total_count == stop_point:
        if time >= stop_time:        
            print('hit in near future',hit_in_near_future_list,hit_in_near_future_accum/total_count)
            print('total ccount',total_count)
            print('hit ratio',hit_rate_list)
            print(round(accum_hit / total_count, 4), 'latency', proactive_latency + reactive_latency)
            print('x axis time',x_time_list)
            print('req rate',req_rate_list)
            print('utilization',utilization_list_reactive)
            # x = range(len(hit_rate_list))
            # plt.plot(x, hit_rate_list)
            # plt.title('LRU caching')
            # plt.legend()
            # plt.show()
            print(total_count)

            return proactive_latency + reactive_latency, hit_rate_list


    print(total_count)
    print(hit_rate_list)
    print (round(accum_hit / total_count,4),'latency',proactive_latency+reactive_latency)
    # x = range(len(hit_rate_list))
    # plt.plot(x, hit_rate_list)
    # plt.title('LRU caching')
    # plt.legend()
    # plt.show()

    return proactive_latency+reactive_latency,hit_rate_list


In [13]:
# for topk_M in [2000]:
#     for size in [50,100,200,500]:
#         for win in [50]:
#             caching_simulation_lfu(CACHE_SIZE = size,LFU_win = win, downloadtime = 0.5, topk_predictive_rank = topk_M, printoutfrequency = 500,begintime = datetime(2010,7,5),stop_time = datetime(2010,7,7))

In [25]:
for topk_M in [2000]:
#     for size in [500,1000,2000,5000]:
    for size in [50,100,200,500,1000]:
        win=100000000
        caching_simulation_lfu(CACHE_SIZE = size,LFU_win = win, downloadtime = 0.5, topk_predictive_rank = topk_M, printoutfrequency = 5000,begintime = datetime(2010,7,5),stop_time = datetime(2010,7,7))

cache size: 50 LFU win 100000000 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
hit in near future 0.0
0.103 proactive download 0 65 buffer content 1
2010-07-05 22:46:40
hit in near future 0.2772
0.075 proactive download 0 69 buffer content 2
2010-07-06 20:32:08
11716
[0.103, 0.075]
0.0884 latency 6309.0
cache size: 100 LFU win 100000000 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
hit in near future 0.0
0.1308 proactive download 0 61 buffer content 1
2010-07-05 22:46:40
hit in near future 0.2772
0.1042 proactive download 0 68 buffer content 2
2010-07-06 20:32:08
11716
[0.1308, 0.1042]
0.1176 latency 6104.5
cache size: 200 LFU win 100000000 downloadtime 0.5 begintime 2010-07-05 00:00:00 stoptime 2010-07-07 00:00:00
hit in near future 0.0
0.1658 proactive download 0 60 buffer content 1
2010-07-05 22:46:40
hit in near future 0.2772
0.147 proactive download 0 63 buffer content 2
2010-07-06 20:32:08
11716
[0.1658, 0.147]
0.1567 la